<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creating-bindings-for-a-custom-type" data-toc-modified-id="Creating-bindings-for-a-custom-type-1">Creating bindings for a custom type</a></span></li><li><span><a href="#Binding-lambda-functions" data-toc-modified-id="Binding-lambda-functions-2">Binding lambda functions</a></span></li><li><span><a href="#Instance-and-static-fields" data-toc-modified-id="Instance-and-static-fields-3">Instance and static fields</a></span></li><li><span><a href="#Dynamic-attributes" data-toc-modified-id="Dynamic-attributes-4">Dynamic attributes</a></span></li><li><span><a href="#Inheritance-and-automatic-upcasting" data-toc-modified-id="Inheritance-and-automatic-upcasting-5">Inheritance and automatic upcasting</a></span></li></ul></div>

## Creating bindings for a custom type

In [1]:
%%file example.cpp 

#include <pybind11/pybind11.h>
namespace py = pybind11;

struct Pet {
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }
    
    std::string name;
};

PYBIND11_MODULE(example, m) {
    py::class_<Pet>(m, "Pet", py::module_local())
        .def(py::init<const std::string &>())
        .def("setName", &Pet::setName)
        .def("getName", &Pet::getName);
}

Overwriting example.cpp


- `class_` creates bindings for a C++ class or struct-style data structure. 
- `init()` is a convenience function that takes the types of a constructor’s parameters as template arguments and wraps the corresponding constructor

In [2]:
!c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` example.cpp -o example`python3-config --extension-suffix`

In [3]:
ls

example2.cpp  example5.cpp  example.cpp
example3.cpp  example6.cpp  example.cpython-36m-x86_64-linux-gnu.so*
example4.cpp  example7.cpp  object-oriented.ipynb


In [4]:
import example

In [5]:
p = example.Pet("Krystal")

In [6]:
p

In [7]:
print(p)

In [8]:
p.getName()

'Krystal'

In [9]:
p.setName('Ripley')

In [10]:
p.getName()

'Ripley'

## Binding lambda functions

In [11]:
%%file example2.cpp 

#include <pybind11/pybind11.h>
namespace py = pybind11;

struct Pet {
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }
    
    std::string name;
};

PYBIND11_MODULE(example2, m) {
    py::class_<Pet>(m, "Pet", py::module_local())
        .def(py::init<const std::string &>())
        .def("setName", &Pet::setName)
        .def("getName", &Pet::getName)
        .def("__repr__",
             [](const Pet &a) {
                 return "<example.Pet named '" + a.name + "'>";
             }
            );
}

Overwriting example2.cpp


In [12]:
!c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` example2.cpp -o example2`python3-config --extension-suffix`

In [13]:
import example2

In [14]:
p = example2.Pet("Krystal")

In [15]:
print(p)

<example.Pet named 'Krystal'>


In [16]:
ls

example2.cpp
example2.cpython-36m-x86_64-linux-gnu.so*
example3.cpp
example4.cpp
example5.cpp
example6.cpp
example7.cpp
example.cpp
example.cpython-36m-x86_64-linux-gnu.so*
object-oriented.ipynb


## Instance and static fields

We can also directly expose the `name` field using the `class_::def_readwrite()` method. A similar `class_::def_readonly()` method also exits for `const` fields. 

In [17]:
%%file example3.cpp 

#include <pybind11/pybind11.h>
namespace py = pybind11;

struct Pet {
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }
    
    std::string name;
};

PYBIND11_MODULE(example3, m) {
    py::class_<Pet>(m, "Pet", py::module_local())
        .def(py::init<const std::string &>())
        .def_readwrite("name", &Pet::name)
        .def("setName", &Pet::setName)
        .def("getName", &Pet::getName)
        .def("__repr__",
             [](const Pet &a) {
                 return "<example.Pet named '" + a.name + "'>";
             }
            );
}

Overwriting example3.cpp


In [18]:
!c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` example3.cpp -o example3`python3-config --extension-suffix`

In [19]:
import example3

In [20]:
p = example3.Pet('Ripley')

In [21]:
print(p)

<example.Pet named 'Ripley'>


In [22]:
p.name 

'Ripley'

In [23]:
p.name = 'Daisy'

In [24]:
print(p)

<example.Pet named 'Daisy'>


Now suppose that `Pet::name` was a private internal variable that can only be accessed via setters and getters.

In this case, the method `class_::def_property()` (`class_::def_property_readonly()` for read-only data) can be used to provide a field-like interface within Python that will transparently call the setter and getter functions:

In [25]:
%%file example4.cpp 

#include <pybind11/pybind11.h>
namespace py = pybind11;

class Pet {
    
public:
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }
private:  
    std::string name;
};

PYBIND11_MODULE(example4, m) {
    py::class_<Pet>(m, "Pet", py::module_local())
        .def(py::init<const std::string &>())
        .def_property("name", &Pet::getName, &Pet::setName)
        .def("setName", &Pet::setName)
        .def("getName", &Pet::getName)
        .def("__repr__",
             [](const Pet &a) {
                 return "<example.Pet named '" + a.getName() + "'>";
             }
            );
}

Overwriting example4.cpp


In [26]:
!c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` example4.cpp -o example4`python3-config --extension-suffix`

In [27]:
import example4

In [28]:
p = example4.Pet('Ripley')

In [29]:
print(p)

<example.Pet named 'Ripley'>


In [30]:
p.name

'Ripley'

## Dynamic attributes

To enable dynamic attributes for C++ classes, the `py::dynamic_attr` tag must be added to the `py::class_ constructor`:



In [31]:
%%file example5.cpp 

#include <pybind11/pybind11.h>
namespace py = pybind11;

class Pet {
    
public:
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }
private:  
    std::string name;
};

PYBIND11_MODULE(example5, m) {
    py::class_<Pet>(m, "Pet", py::module_local(), py::dynamic_attr())
        .def(py::init<const std::string &>())
        .def_property("name", &Pet::getName, &Pet::setName)
        .def("setName", &Pet::setName)
        .def("getName", &Pet::getName)
        .def("__repr__",
             [](const Pet &a) {
                 return "<example.Pet named '" + a.getName() + "'>";
             }
            );
}

Overwriting example5.cpp


In [32]:
!c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` example5.cpp -o example5`python3-config --extension-suffix`

In [33]:
import example5

In [34]:
p = example5.Pet('Ripley')

In [35]:
p.name

'Ripley'

In [36]:
p.age = 6 # OK, dynamically add a new attribute

In [37]:
print(p)

<example.Pet named 'Ripley'>


In [38]:
p.age

6

In [39]:
p.name = 'Bob'

In [40]:
p.__dict__ # Just like a native Python class

{'age': 6}

Note that there is a small runtime cost for a class with dynamic attributes. Not only because of the addition of a `__dict__`, but also because of more expensive garbage collection tracking which must be activated to resolve possible circular references. Native Python classes incur this same cost by default, so this is not anything to worry about. By default, pybind11 classes are more efficient than native Python classes. Enabling dynamic attributes just brings them on par.



## Inheritance and automatic upcasting

There are two different ways of indicating a hierarchical relationship to pybind11: 
- the first specifies the C++ base class as an extra template parameter of the `class_`:

In [41]:
%%file example6.cpp 

#include <pybind11/pybind11.h>
namespace py = pybind11;

struct Pet {
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }
    
    std::string name;
};


struct Dog : Pet {
    Dog(const std::string &name) : Pet(name) { }
    std::string bark() const { return "woof!"; }
};

PYBIND11_MODULE(example6, m) {
    py::class_<Pet>(m, "Pet", py::module_local(), py::dynamic_attr())
        .def(py::init<const std::string &>())
        .def_readwrite("name", &Pet::name);
    
    // Method 1: template paramter:
    py::class_<Dog, Pet /* <- specify C++ parent type */>(m, "Dog")
        .def(py::init<const std::string &>())
        .def("bark", &Dog::bark);
}

Overwriting example6.cpp


In [42]:
!c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` example6.cpp -o example6`python3-config --extension-suffix`

In [43]:
import example6

In [44]:
p = example6.Dog('Molly')

In [45]:
p.name

'Molly'

In [46]:
p.bark()

'woof!'

- Alternatively, we can also assign a name to the previously bound Pet class_ object and reference it when binding the Dog class:

In [47]:
%%file example7.cpp 

#include <pybind11/pybind11.h>
namespace py = pybind11;

struct Pet {
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }
    
    std::string name;
};


struct Dog : Pet {
    Dog(const std::string &name) : Pet(name) { }
    std::string bark() const { return "woof!"; }
};

PYBIND11_MODULE(example7, m) {
    py::class_<Pet> pet(m, "Pet", py::module_local(), py::dynamic_attr());
        pet.def(py::init<const std::string &>())
        .def_readwrite("name", &Pet::name);
    
    // Method 2: pass parent class_ object
    py::class_<Dog>(m, "Dog", pet, py::module_local(), py::dynamic_attr() /* <- specify C++ parent type */)
        .def(py::init<const std::string &>())
        .def("bark", &Dog::bark);
}

Overwriting example7.cpp


In [48]:
!c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` example7.cpp -o example7`python3-config --extension-suffix`

In [49]:
import example7

In [50]:
p = example7.Dog('Bob')

In [51]:
print(p)

In [52]:
p.name

'Bob'

In [53]:
p.bark()

'woof!'